# Project "Know_Your_Rent_Muc": Part Three
# Prediction Of The Rent in Munich

At this part the focus will be to find the best model for my use case. The score is defined as the R2 value and is the correlation between the predicted values and the true values of the test date.

In [103]:
# Loading necessary modules
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score,KFold, GridSearchCV
from sklearn.metrics import classification_report, r2_score, mean_squared_error
import scipy.stats as stats
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve, ShuffleSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

## 1. Data Loading and Selecting

First of all, I load the wrangled data.

In [104]:
df = pd.read_csv("wrangled_data.csv") # Load the data in a dataframe

In [105]:
df.head() # Show the first 5 rows of the dataset

,object_area,object_price,object_rooms,Balkon,EBK,Garten,object_district_Allach-Untermenzing,object_district_Altstadt-Lehel,object_district_Au-Haidhausen,object_district_Aubing-Lochhausen-Langwied,...,object_district_Untergiesing-Harlaching,object_objektart_sub_APARTMENT,object_objektart_sub_DACHGESCHOSS,object_objektart_sub_ERDGESCHOSS,object_objektart_sub_ETAGE,object_objektart_sub_LOFT-STUDIO-ATELIER,object_objektart_sub_MAISONETTE,object_objektart_sub_PENTHOUSE,object_objektart_sub_SOUTERRAIN,object_objektart_sub_Wohnung
0,3.610918,1120,0.000000,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,3.971423,1339,0.693147,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,4.161380,1509,0.693147,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,3.901164,1600,0.693147,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,4.566741,2085,1.386294,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In this step the data get splitted in the x-data and the y-data for the model.

In [106]:
y = df['object_price'] # select the target column
X = df.drop(['object_price'],axis= 1) # select the features

## 2. Modeling

The goal is to predict the rent.  That's why it's a regression problem.

## 2.1. Linear Regression

Now I definie the Machine Learning Models. I start with the most simplest one: The linear regression. 

In [107]:
lr = LinearRegression() # load the machine learning model

In [124]:
cv = ShuffleSplit(n_splits=5, test_size=0.25, random_state=30) # Shuffeling the data for the cross validation

In the next part I set the cross validation program. This is more accurate to test the model performence then a classic train/test split. 

In [125]:
# perform cross validation and print the results and the mean of the results
scores_lr = cross_val_score(lr, X, y, cv=cv, scoring='r2')
print(scores_lr)
print('---------')
print(np.mean(scores_lr))

[0.58509565 0.70078208 0.62534314 0.6230129  0.58571552]
---------
0.6239898583910228


The value is ok, but there are other models which can have a better performence. The next models are:
- Ridge
- Lasso
- Regression Randowm Forest
- Regressio Gradient Boosting

## 2.2. Ridge Regression

In [127]:
ridge = Ridge()  # load the machine learning model

In [128]:
# define the range of parameter values
param_grid = {
    'alpha': list(range(30))
}

# search for the best parameter
grid = GridSearchCV(ridge, param_grid, cv=cv)
grid.fit(X, y)

# print the best parameter
print("Best parameter: ", grid.best_params_)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=8.19449e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Best parameter:  {'alpha': 3}


In [131]:
ridge = Ridge(alpha=3)  # load the machine learning model with the optimal parameter

In [133]:
# perform cross validation and print the results and the mean of the results
scores_ridge = cross_val_score(ridge, X, y, cv=cv, scoring='r2')
print(scores_ridge)
print('-----------')
print(np.mean(scores_ridge))

[0.5794855  0.72267528 0.63254944 0.64214445 0.58907343]
-----------
0.633185621671551


The result is a little bit better.

## 2.3. Lasso Regression

In [18]:
lasso = Lasso() # load the machine learning model

In [138]:
# define the range of parameter values
param_grid = {
    'alpha': list(range(30))
}

# search for the best parameter
grid = GridSearchCV(lasso, param_grid, cv=cv)
grid.fit(X, y)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.001e+07, tolerance: 5.767e+04 Linear regression models with null weight for the l1 regularization term are more ef

GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=30, test_size=0.25, train_size=None),
             estimator=Lasso(alpha=3),
             param_grid={'alpha': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                   14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
                                   25, 26, 27, 28, 29]})

In [137]:
print("Best parameter: ", grid.best_params_) # print the best parameter

Best parameter:  {'alpha': 3}


In [135]:
lasso = Lasso(alpha=3) # load the machine learning model

In [136]:
# perform cross validation and print the results and the mean of the results
scores_lasso = cross_val_score(lasso, X, y, cv=cv, scoring='r2')
print(scores_lasso)
print('-----------')
print(np.mean(scores_lasso))

[0.58248603 0.7215055  0.63059388 0.64426251 0.59858636]
-----------
0.6354868560198093


The improvement is small and not significant. 

## 2.4. Random Forest

In [22]:
rf_regr = RandomForestRegressor() # load the machine learning model

In [140]:
# define the range of parameter values
param_grid = {
    'n_estimators': [10, 50, 100, 200, 300, 400, 500],
    'max_depth': [1, 5, 10, 15, 20, 30, 40, 50],
}

# search for the best parameter
grid = GridSearchCV(rf_regr , param_grid, cv=cv)
grid.fit(X, y)

# print the best parameter
print("Best parameter: ", grid.best_params_)

Best parameter:  {'max_depth': 40, 'n_estimators': 50}


In [141]:
# load the machine learning model with the optimal parameter
rf_regr = RandomForestRegressor(n_estimators=50, max_depth=40) 

In [142]:
# perform cross validation and print the results and the mean of the results
scores_rf = cross_val_score(rf_regr, X_1, y, cv=cv, scoring='r2')
print(scores_rf)
print('-----------')
print(np.mean(scores_rf))

[0.76543326 0.79759586 0.78449662 0.78688858 0.78248831]
-----------
0.7833805268852169


## 2.5 Gradient Boosting

In [80]:
gb_regr = GradientBoostingRegressor() # load the machine learning model

In [143]:
# define the range of parameter values
param_grid = {
    'n_estimators': [50, 100, 200,300,400],
    'max_depth': [1, 3, 5],
    'learning_rate': [0.5, 0.1, 0.05, 0.01, 0.001],
}

# search for the best parameter
grid = GridSearchCV(gb_regr , param_grid, cv=cv)
grid.fit(X, y)

# print the best parameter
print("Best parameter: ", grid.best_params_)

Best parameter:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}


In [147]:
# load the machine learning model with the optimal parameter
gb_regr = GradientBoostingRegressor(learning_rate=0.1, max_depth=3, n_estimators=50)

In [148]:
# perform cross validation and print the results and the mean of the results
scores_r2 = cross_val_score(gb_regr, X, y, cv=cv, scoring='r2')
print(scores_r2)
print('-----------')
print(np.mean(scores_r2))

[0.77104768 0.80455444 0.8120158  0.7438007  0.80243561]
-----------
0.7867708450813771


All relevant algorithm have a R2-Score from 0.623 to 0.787. The best of them is the Gradient Boosting. 